## Perceptron POStagger

In this assignment, you will need to treat postagging on one word as classification problem.
The classifier we choose is the (simplest) perceptron model.
You will need to a perceptron models that _classify on current window of words, along with previous POStag._

You will need to implement the following parts:
- mapping facility from feature string to its index in the parameter matrix W
- perceptron update
- calculate the score of given features and class
- (part of) the feature extraction

You should get an accuracy of more than **91.0** after the first round of training on the development set.

In [1]:
from copy import copy
from collections import Counter

Min_Times = 5

def not_rare_set(dataset):
    counter = Counter()
    for data_x, data_y in dataset:
        counter.update(data_x)
    return counter
    

class PerceptronClassifier(object):
    # The perceptron classifier
    def __init__(self, max_iter=10, training_data=None, devel_data=None):
        '''
        Parameters
        ----------
        max_iter: int
            The max number of iteration
        training_data: list
            The training data
        devel_data: list
            The development data, to determine the best iteration.
        '''
        self.max_iter = max_iter
        if training_data is not None:
            self.fit(training_data, devel_data)

            
    def fit(self, training_data, devel_data=None):
        '''
        Estimate the parameters for perceptron model. For multi-class perceptron, parameters can be
        treated as a T \times D matrix W, where T is the number of labels and D is the number of
        features.
        '''
        # feature_alphabet is a mapping from feature string to it's dimension in the feature space,
        # e.g. feature_alphabet['U1=I']=3, which means 'U1=I' is in the third column of W
        # 
        # W = [[ . . 1 . . .],
        #      ...
        #      [ . . 1 . . .]]
        #            ^
        #            |
        #         'U1=I'
        self.feature_alphabet = {'None': 0}
        self.label_alphabet = {}

        # Extract features, build the feature_alphabet, label_alphabet and training instance pairs.
        # Each instance consist a tuple (X, Y) where X is the mapped features (list(int)), and Y is
        # the index of the corresponding label.
        instances = []
        next_label_index = len(self.label_alphabet)
        self.next_feature_index = len(self.feature_alphabet)
        self.not_rare = not_rare_set(training_data)
        for words, tags in training_data:
            L = len(words)
            prev = ['<s>', '<s>']
            for i in range(L):
                # Your code here, extract features and give it into X, convert POStag to index and
                # give it to Y
                if self.not_rare[words[i]] < Min_Times:
                    is_rare = True
                else:
                    is_rare = False
                X = self.extract_features(words, i, prev, add=True, is_rare=is_rare)
                if tags[i] not in self.label_alphabet:
                    self.label_alphabet[tags[i]] = next_label_index
                    Y = next_label_index
                    next_label_index += 1
                else:
                    Y = self.label_alphabet[tags[i]]
                instances.append((X, Y))
                prev[0] = prev[1]
                prev[1] = tags[i]

        # Build a mapping from index to label string to recover POStags.
        self.labels = [-1 for k in self.label_alphabet]
        for k in self.label_alphabet:
            self.labels[self.label_alphabet[k]] = k

        self.D, self.T = len(self.feature_alphabet), len(self.label_alphabet)
        print('number of features : %d' % self.D)
        print('number of labels: %d' % self.T)

        # Allocate the weight matrix W
        self.W = [[0 for j in range(self.D)] for i in range(self.T)]
        self.best_W = None
        best_acc = None

        devel_not_rare = not_rare_set(devel_data)
        for it in range(self.max_iter):
            # The training part,
            n_errors = 0
            print('training iteration #%d' % it)
            for X, Y in instances:
                # Your code here, ake a prediction and give it to Z
                Z = self._predict(X) #is _predict not predict
                if Z != Y:
                    # Your code here. If the predict is incorrect, perform the perceptron update
                    n_errors += 1
                    for x in X:
                        # The perceptron update part.
                        self.W[Y][x] += 1
                        self.W[Z][x] -= 1

            print('training error %d' % n_errors)

            if devel_data is not None:
                # Test accuracy on the development set if provided.
                n_corr, n_total = 0, 0
                for words, tags in devel_data:
                    prev = ['<s>', '<s>']
                    for i in range(len(words)):
                        if devel_not_rare[words[i]] < Min_Times:
                            is_rare = True
                        else:
                            is_rare = False
                        Z = self.predict(words, i, prev, is_rare)
                        Y = self.label_alphabet[tags[i]]
                        if Z == Y:
                            n_corr += 1
                        n_total += 1
                        prev[0] = prev[1]
                        prev[1] = self.labels[Z]
                print('accuracy: %f' % (float(n_corr)/n_total))
                if best_acc < float(n_corr)/n_total:
                    # If this round is better than before, save it.
                    best_acc = float(n_corr)/n_total
                    self.best_W = copy(self.W)
                    
        if self.best_W is None:
            self.best_W = copy(self.W)

            
    def extract_features(self, words, i, prev_tag=None, add=True, is_rare=False):
        '''
        Extract features from words and prev POS tag, if `add` is True, also insert the feature
        string to the feature_alphabet.
        
        Parameters
        ----------
        words: list(str)
            The words list  
        i: int
            The position
        prev_tag: str
            Previous POS tag
        add: bool
            If true, insert the feature to feature_alphabet.
            
        Return
        ------
        mapped_features: list(int)
            The list of hashed features.
        '''
        L = len(words)
        context = ['<s>' if i- 2 < 0 else words[i- 2],
                   '<s>' if i- 1 < 0 else words[i- 1],
                   words[i],
                   '<e>' if i+ 1 >= L else words[i+ 1],
                   '<e>' if i+ 2 >= L else words[i+ 1]]
        raw_features = ['U2=%s' % context[1],
                        'U3=%s' % context[2],
                        'U4=%s' % context[3],
                        'U5=%s' % context[4],
                        ]
        if prev_tag is not None:
            raw_features.append('T2=%s' % prev_tag[1])
            raw_features.append('T1,2=%s/%s' % (prev_tag[0], prev_tag[1]))
        if not is_rare:
            raw_features.append('U1=%s' % context[0])
        else:
            wl = len(words[i])
            extent = 4
            if wl < 4:
                extent = wl
            for j in range(extent):
                raw_features.append('PRE=%s' % (words[i][:j+1]))
            for j in range(wl-1, wl-1-extent, -1):
                raw_features.append('SUF=%s' % (words[i][j:]))
            for j in range(wl):
                if words[i][j] >= '0' and words[i][j] <= '9':
                    raw_features.append('NUM')
                    break
            for j in range(wl):
                if words[i][j] >= 'A' and words[i][j] <= 'Z':
                    raw_features.append('UPPER')
                    break
            for j in range(wl):
                if words[i][j] >= '-':
                    raw_features.append('HYPHEN')
                    break

        mapped_features = []
        for f in raw_features:
            if add and (f not in self.feature_alphabet):
                # Your code here, insert the feature string to the feature_alphabet.
                self.feature_alphabet[f] = self.next_feature_index
                self.next_feature_index += 1
            # Your code here, map the string feature to index.
            if f in self.feature_alphabet:
                mapped_features.append(self.feature_alphabet[f])
            
        return mapped_features

    
    def _score(self, features, t):
        '''
        Calcuate score from the given features and label t
        
        Parameters
        ----------
        features: list(int)
            The hashed features
        t: int
            The index of label
            
        Return
        ------
        s: int
            The score
        '''
        # Your code here, compute the score.
        s = 0
        for i in features:
            s += self.W[t][i]
        
        return s

    
    def _predict(self, features):
        '''
        Calcuate score from the given features and label t
        
        Parameters
        ----------
        features: list(int)
            The hashed features
        t: int
            The index of label
            
        Return
        ------
        best_y: int
            The highest scored label's index
        '''
        pred_scores = [self._score(features, y) for y in range(self.T)]

        best_score, best_y = None, None
        # Your code here, find the highest scored class from pred_scores
        for y in range(self.T):
            if best_score < pred_scores[y]:
                best_score = pred_scores[y]
                best_y = y
        
        return best_y
    
    
    def predict(self, words, i, prev_tag=None, is_rare=False):
        '''
        Make prediction on list of words
        
        Parameters
        ----------
        words: list(str)
            The words list  
        i: int
            The position
        prev_tag: str
            Previous POS tag
        
        Return
        ------
        y: int
            The predicted label's index
        '''
        X = self.extract_features(words, i, prev_tag, False, is_rare)
        y = self._predict(X)
        return y

In [2]:
def greedy_search(words, classifier, counter):
    '''
    Perform greedy search on the classifier.
    
    Parameters
    ----------
    words: list(str)
        The word list
    classifier: PerceptronClassifier
        The classifier object.
    '''
    prev = ['<s>', '<s>']
    ret=[]
    for i in range(len(words)):
        # Your code here, implement the greedy search,
        if counter[words[i]] < Min_Times:
            is_rare = True
        else:
            is_rare = False
        label_index = classifier.predict(words, i, prev, is_rare)
        label = classifier.labels[label_index] #attention: label should be a string not index
        ret.append(label)
        prev[0] = prev[1]
        prev[1] = ret[-1]
    return ret

In [3]:
from dataset import read_dataset

train_dataset = read_dataset('./penn.train.pos.gz')
devel_dataset = read_dataset('./penn.devel.pos.gz')

print('%d is training sentences.' % len(train_dataset))
print('%d is development sentences.' % len(devel_dataset))

perceptron = PerceptronClassifier(max_iter=5, training_data=train_dataset, devel_data=devel_dataset)

#n_corr, n_total = 0, 0
#for devel_data in devel_dataset:
#    devel_data_x, devel_data_y = devel_data
#    pred_y = greedy_search(devel_data_x, perceptron)
#    for pred_tag, corr_tag in zip(pred_y, devel_data_y):
#        if pred_tag == corr_tag:
#            n_corr += 1
#        n_total += 1
#print("accuracy: %f" % (float(n_corr)/ n_total))

39832 is training sentences.
1700 is development sentences.
number of features : 253334
number of labels: 45
training iteration #0
training error 88534
accuracy: 0.932223
training iteration #1
training error 40915
accuracy: 0.936112
training iteration #2
training error 30989
accuracy: 0.941471
training iteration #3
training error 26023
accuracy: 0.941920
training iteration #4
training error 22840
accuracy: 0.941496


In [4]:
tmp_counter = Counter()
for k in range(Min_Times):
    tmp_counter.update(['HMM', 'is', 'a', 'widely', 'used', 'model', '.'])
    tmp_counter.update(['I', 'like', 'cat', ',', 'but', 'I', 'hate', 'eating', 'fish', '.'])
print greedy_search(['HMM', 'is', 'a', 'widely', 'used', 'model', '.'], perceptron, tmp_counter)
print greedy_search(['I', 'like', 'cat', ',', 'but', 'I', 'hate', 'eating', 'fish', '.'], perceptron, tmp_counter)

['NNP', 'VBZ', 'DT', 'RB', 'JJ', 'NN', '.']
['PRP', 'VBP', 'RB', ',', 'CC', 'PRP', 'VBP', 'VBG', 'NN', '.']


In [5]:
# Work around the test dataset
from __future__ import print_function

test_dataset = read_dataset('./penn.test.pos.blind.gz')

fpo=open('./penn.test.perceptron.pos.out', 'w')

test_not_rare = not_rare_set(test_dataset)
for test_data_x, test_data_y in test_dataset:
    pred_y = greedy_search(test_data_x, perceptron, test_not_rare)
    print(" ".join(y for y in pred_y), file=fpo)

fpo.close()